In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text Summarization of Large Documents using LangChain 🦜🔗

## Overview

Text summarization is an NLP task that creates a concise and informative summary of a longer text. LLMs can be used to create summaries of news articles, research papers, technical documents, and other types of text.

Summarizing large documents can be challenging. To create summaries, you need to apply summarization strategies to your indexed documents. You have already seen some of these strategies in the previous notebooks. If you haven't completed it, it is recommended to do so to have a basic understanding of how to summarize large documents.

In this notebook, you will use LangChain, a framework for developing LLM applications, to apply some summarization strategies. The notebook covers several examples of how to summarize large documents.

### Objective

In this tutorial, you learn how to use LangChain with PaLM API to summarize large documents by working through the following examples:

- Stuffing method
- MapReduce method
- Refine method

### Costs

This tutorial uses billable components of Google Cloud:
- Vertex AI Generative AI Studio

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Getting Started

### SKIP: Install Vertex AI SDK & Other dependencies

Skip pip install for hackathon since genai conda env has all required packages

In [ ]:
# !sudo apt -y -qq install tesseract-ocr
# !sudo apt -y -qq install libtesseract-dev
# !sudo apt-get -y -qq install poppler-utils #required by PyPDF2 for page count and other pdf utilities
# !sudo apt-get -y -qq install python-dev libxml2-dev libxslt1-dev antiword unrtf poppler-utils tesseract-ocr flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig

In [ ]:
# import os

# if not os.getenv("IS_TESTING"):
#     USER = "--user"
# else:
#     USER = ""

# ! pip3 install {USER} --upgrade pytesseract pypdf PyPDF2 textract langchain transformers google-cloud-aiplatform --quiet

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

Uncomment the following cell when not on Vertex AI Workbench.

In [ ]:
# PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
# REGION = "us-central1"

# import vertexai

# vertexai.init(project=PROJECT_ID, location=REGION)

### Import libraries

In [3]:
import urllib
import warnings
from pathlib import Path as p

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader
from langchain.llms import VertexAI

warnings.filterwarnings("ignore")

### Import models

You load the pre-trained text generation model called `text-bison@001`.

In [4]:
vertex_llm_text = VertexAI(model_name="text-bison@001")

## Summarization with Large Documents

### Preparing data files

To begin, you will need to download a few files that are required for the summarizing tasks below.

In [5]:
data_folder = p.cwd() / "data"
p(data_folder).mkdir(parents=True, exist_ok=True)

pdf_url = "https://services.google.com/fh/files/misc/practitioners_guide_to_mlops_whitepaper.pdf"
pdf_file = str(p(data_folder, pdf_url.split("/")[-1]))

urllib.request.urlretrieve(pdf_url, pdf_file)

('/home/jupyter/vertex-genai-hackathon/use-cases/document-summarization/data/practitioners_guide_to_mlops_whitepaper.pdf',
 <http.client.HTTPMessage at 0x7f9ce7f89cf0>)

### Extract text from the PDF

You use an `PdfReader` to extract the text from our scanned documents.

In [6]:
pdf_loader = PyPDFLoader(pdf_file)
pages = pdf_loader.load_and_split()
print(pages[3].page_content)

4
Organizations can use the framework to identify gaps in building an integrated ML platform and to focus on the scale 
and automate themes from Google’s AI Adoption Framework. The decision about whether (or to which degree) to 
adopt each of these processes and capabilities in your organization depends on your business context. For exam -
ple, you must determine the business value that the framework creates when compared to the cost of purchasing or 
building capabilities (for example, the cost in engineering hours).
Overview of MLOps lifecycle  
and core capabilities
Despite the growing recognition of AI/ML as a crucial pillar of digital transformation, successful deployments and 
effective operations are a bottleneck for getting value from AI. Only one in two organizations has moved beyond 
pilots and proofs of concept. Moreover, 72% of a cohort of organizations that began AI pilots before 2019 have not 
been able to deploy even a single application in production.1 Algorithmia’s sur

## Method 1: Stuffing

Stuffing is the simplest method to pass data to a language model. It "stuffs" text into the prompt as context in a way that all of the relevant information can be processed by the model to get what you want.

In LangChain, you can use `StuffDocumentsChain` as part of the `load_summarize_chain` method. What you need to do is setting `stuff` as `chain_type` of your chain.

### Prompt design with `Stuffing` chain

In [7]:
prompt_template = """Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
  """

prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

### Retrying
Initiate a chain using `stuff` method and process three pages document.

In [8]:
stuff_chain = load_summarize_chain(vertex_llm_text, chain_type="stuff", prompt=prompt)

In [9]:
three_pages = pages[:3]

In [10]:
three_pages

[Document(page_content='Practitioners guide to MLOps:  \nA framework for continuous \ndelivery and automation of  \nmachine learning.White paper\nMay 2021\nAuthors:  \nKhalid Salama,  \nJarek Kazmierczak,  \nDonna Schut', metadata={'source': '/home/jupyter/vertex-genai-hackathon/use-cases/document-summarization/data/practitioners_guide_to_mlops_whitepaper.pdf', 'page': 0}),
 Document(page_content='Table of Contents\nExecutive summary  3\nOverview of MLOps lifecycle and core capabilities  4\nDeep dive of MLOps processes  15\nPutting it all together  34\nAdditional resources  36Building an ML-enabled system  6\nThe MLOps lifecycle  7\nMLOps: An end-to-end workflow  8\nMLOps capabilities  9\n      Experimentation  11\n      Data processing  11\n      Model training  11\n      Model evaluation  12\n      Model serving  12\n      Online experimentation  13\n      Model monitoring  13\n      ML pipelines  13\n      Model registry  14\n      Dataset and feature repository  14\n      ML metada

In [11]:
try:
    print(stuff_chain.run(three_pages))
except Exception as e:
    print(
        "The code failed since it won't be able to run inference on such a huge context and throws this exception: ",
        e,
    )

- MLOps is a set of standardized processes and technology capabilities for building, deploying, and operationalizing ML systems rapidly and reliably.
   - MLOps is important for successful adoption of ML-based systems because it can help organizations improve collaboration between teams, improve the reliability and scalability of ML systems, and shorten development cycle times.
   - The MLOps lifecycle consists of five stages: ML development, training operationalization, continuous training, model deployment, and prediction serving.
   - MLOps capabilities include experimentation, data processing, model training, model evaluation, model serving, online experimentation, model monitoring, ML pipelines,


As you can see, with the `stuff` method, you can summarize the entire document content with a single API call passing all data at once.

Depending on the context length of LLM, the `stuff` method would not work as it result in a prompt larger than the context length.

In [12]:
try:
    print(stuff_chain.run(pages))
except Exception as e:
    print(
        "The code failed since it won't be able to run inference on such a huge context and throws this exception: ",
        e,
    )

The code failed since it won't be able to run inference on such a huge context and throws this exception:  400 The model supports up to 8192 input tokens, but received 13326 tokens.


As expected, the code returns the expection message.

### Considerations

The `stuffing` method is a way to summarize text by feeding the entire document to a large language model (LLM) in a single call. This method has both pros and cons.

The stuffing method only requires a single call to the LLM, which can be faster than other methods that require multiple calls. When summarizing text, the LLM has access to all the data at once, which can result in a better summary.

But, LLMs have a context length, which is the maximum number of tokens that can be processed in a single call. If the document is longer than the context length, the stuffing method will not work. Also the stuffing method is not suitable for summarizing large documents, as it can be slow and may not produce a good summary.

Let's explore other approaches to help deal with having longer text than context lengh limit of LLMs.

## Method 2: MapReduce

The `MapReduce` method implements a multi-stage summarization. It is a technique for summarizing large pieces of text by first summarizing smaller chunks of text and then combining those summaries into a single summary.

In LangChain, you can use `MapReduceDocumentsChain` as part of the `load_summarize_chain` method. What you need to do is setting `map_reduce` as `chain_type` of your chain.

### Prompt design with `MapReduce` chain

In our example, you have a 32-page document that you need to summarize.

With LangChain, the `map_reduce` chain breaks the document down into 1024 token chunks max. Then it runs the initial prompt you define on each chunk to generate a summary of that chunk. In the example below, you use the following first stage or map prompt.

```Write a concise summary of the following text delimited by triple backquotes. Return your response in bullet points which covers the key points of the text.
'''{text}'''. BULLET POINT SUMMARY:```

Once summaries for all of the chunks are generated, it runs a different prompt to combine those summaries into a single summary. In the example below, you use the following second stage or combine prompt.

```Write a summary of the entire document that includes the main points from all of the individual summaries.```

In [13]:
map_prompt_template = """
                      Write a summary of this chunk of text that includes the main points and any important details.
                      {text}
                      """

map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

combine_prompt_template = """
                      Write a concise summary of the following text delimited by triple backquotes.
                      Return your response in bullet points which covers the key points of the text.
                      ```{text}```
                      BULLET POINT SUMMARY:
                      """

combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["text"]
)

### Generate summaries using MapReduce method

After defining prompts, you initialize the associated `map_reduce_chain`.

In [14]:
map_reduce_chain = load_summarize_chain(
    vertex_llm_text,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=True,
)

Then, you generate summaries using the chain. Notice that Langchain use a tokenizer (from transformer library) with 1024 token limit by default.

In [15]:
map_reduce_outputs = map_reduce_chain({"input_documents": pages})

Token indices sequence length is longer than the specified maximum sequence length for this model (4084 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1115 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1246 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1248 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1382 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence leng

After summaries are generated, you can validate them by organize input documents and associated output in a Pandas Dataframe.

In [16]:
final_mp_data = []
for doc, out in zip(
    map_reduce_outputs["input_documents"], map_reduce_outputs["intermediate_steps"]
):
    output = {}
    output["file_name"] = p(doc.metadata["source"]).stem
    output["file_type"] = p(doc.metadata["source"]).suffix
    output["page_number"] = doc.metadata["page"]
    output["chunks"] = doc.page_content
    output["concise_summary"] = out
    final_mp_data.append(output)

In [17]:
pdf_mp_summary = pd.DataFrame.from_dict(final_mp_data)
pdf_mp_summary = pdf_mp_summary.sort_values(
    by=["file_name", "page_number"]
)  # sorting the dataframe by filename and page_number
pdf_mp_summary.reset_index(inplace=True, drop=True)
pdf_mp_summary.head()

,file_name,file_type,page_number,chunks,concise_summary
0,practitioners_guide_to_mlops_whitepaper,.pdf,0,Practitioners guide to MLOps: \nA framework f...,This white paper provides a guide for practiti...
1,practitioners_guide_to_mlops_whitepaper,.pdf,1,Table of Contents\nExecutive summary 3\nOverv...,"This document provides an overview of MLOps, i..."
2,practitioners_guide_to_mlops_whitepaper,.pdf,2,"Executive summary\nAcross industries, DevOps a...","This document provides an overview of MLOps, w..."
3,practitioners_guide_to_mlops_whitepaper,.pdf,3,4\nOrganizations can use the framework to iden...,- MLOps is a framework that helps organization...
4,practitioners_guide_to_mlops_whitepaper,.pdf,4,5\nprocesses in place is one of the differenti...,


In [18]:
index = 3
print("[Context]")
print(pdf_mp_summary["chunks"].iloc[index])
print("\n\n [Simple Summary]")
print(pdf_mp_summary["concise_summary"].iloc[index])
print("\n\n [Page number]")
print(pdf_mp_summary["page_number"].iloc[index])
print("\n\n [Source: file_name]")
print(pdf_mp_summary["file_name"].iloc[index])

[Context]
4
Organizations can use the framework to identify gaps in building an integrated ML platform and to focus on the scale 
and automate themes from Google’s AI Adoption Framework. The decision about whether (or to which degree) to 
adopt each of these processes and capabilities in your organization depends on your business context. For exam -
ple, you must determine the business value that the framework creates when compared to the cost of purchasing or 
building capabilities (for example, the cost in engineering hours).
Overview of MLOps lifecycle  
and core capabilities
Despite the growing recognition of AI/ML as a crucial pillar of digital transformation, successful deployments and 
effective operations are a bottleneck for getting value from AI. Only one in two organizations has moved beyond 
pilots and proofs of concept. Moreover, 72% of a cohort of organizations that began AI pilots before 2019 have not 
been able to deploy even a single application in production.1 Algorit

### Considerations

With `MapReduce` method, the model is able to summarize a large paper by overcoming the context limit of `Stuffing` method with parallel processing.

However, the `MapReduce` requires multiple calls to the model and potentially losing context between pages.

To deal this challenge, you can try another method to summarize multiple pages at a time.

## Method 3: Refine

The Refine method is an alternative method to deal with large document summarization. It works by first running an initial prompt on a small chunk of data, generating some output. Then, for each subsequent document, the output from the previous document is passed in along with the new document, and the LLM is asked to refine the output based on the new document.

In LangChain, you can use `MapReduceDocumentsChain` as part of the load_summarize_chain method. What you need to do is setting `refine` as `chain_type` of your chain.

### Prompt design with `Refine` chain

With LangChain, the `refine` chain requires two prompts.

The question prompt to generate the output for subsequent task. The refine prompt to refine the output based on the generated content.

In this example, the question prompt is:

```
Please provide a summary of the following text.
TEXT: {text}
SUMMARY:
```

and the refine prompt is:

```
Write a concise summary of the following text delimited by triple backquotes.
Return your response in bullet points which covers the key points of the text.
```{text}```
BULLET POINT SUMMARY:
```


In [19]:
question_prompt_template = """
                  Please provide a summary of the following text.
                  TEXT: {text}
                  SUMMARY:
                  """

question_prompt = PromptTemplate(
    template=question_prompt_template, input_variables=["text"]
)

refine_prompt_template = """
              Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
              """

refine_prompt = PromptTemplate(
    template=refine_prompt_template, input_variables=["text"]
)

### Generate summaries using Refine method

After you define prompts, you initiate a summarization chain using `refine` chain type.

In [20]:
refine_chain = load_summarize_chain(
    vertex_llm_text,
    chain_type="refine",
    question_prompt=question_prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
)

Then, you use the summatization chain to summarize document using Refine method.

In [21]:
refine_outputs = refine_chain({"input_documents": pages})

Below you can see the resulting summaries.

In [22]:
final_refine_data = []
for doc, out in zip(
    refine_outputs["input_documents"], refine_outputs["intermediate_steps"]
):
    output = {}
    output["file_name"] = p(doc.metadata["source"]).stem
    output["file_type"] = p(doc.metadata["source"]).suffix
    output["page_number"] = doc.metadata["page"]
    output["chunks"] = doc.page_content
    output["concise_summary"] = out
    final_refine_data.append(output)

In [23]:
pdf_refine_summary = pd.DataFrame.from_dict(final_refine_data)
pdf_refine_summary = pdf_mp_summary.sort_values(
    by=["file_name", "page_number"]
)  # sorting the datafram by filename and page_number
pdf_refine_summary.reset_index(inplace=True, drop=True)
pdf_refine_summary.head()

,file_name,file_type,page_number,chunks,concise_summary
0,practitioners_guide_to_mlops_whitepaper,.pdf,0,Practitioners guide to MLOps: \nA framework f...,This white paper provides a guide for practiti...
1,practitioners_guide_to_mlops_whitepaper,.pdf,1,Table of Contents\nExecutive summary 3\nOverv...,"This document provides an overview of MLOps, i..."
2,practitioners_guide_to_mlops_whitepaper,.pdf,2,"Executive summary\nAcross industries, DevOps a...","This document provides an overview of MLOps, w..."
3,practitioners_guide_to_mlops_whitepaper,.pdf,3,4\nOrganizations can use the framework to iden...,- MLOps is a framework that helps organization...
4,practitioners_guide_to_mlops_whitepaper,.pdf,4,5\nprocesses in place is one of the differenti...,


In [24]:
index = 3
print("[Context]")
print(pdf_refine_summary["chunks"].iloc[index])
print("\n\n [Simple Summary]")
print(pdf_refine_summary["concise_summary"].iloc[index])
print("\n\n [Page number]")
print(pdf_refine_summary["page_number"].iloc[index])
print("\n\n [Source: file_name]")
print(pdf_refine_summary["file_name"].iloc[index])

[Context]
4
Organizations can use the framework to identify gaps in building an integrated ML platform and to focus on the scale 
and automate themes from Google’s AI Adoption Framework. The decision about whether (or to which degree) to 
adopt each of these processes and capabilities in your organization depends on your business context. For exam -
ple, you must determine the business value that the framework creates when compared to the cost of purchasing or 
building capabilities (for example, the cost in engineering hours).
Overview of MLOps lifecycle  
and core capabilities
Despite the growing recognition of AI/ML as a crucial pillar of digital transformation, successful deployments and 
effective operations are a bottleneck for getting value from AI. Only one in two organizations has moved beyond 
pilots and proofs of concept. Moreover, 72% of a cohort of organizations that began AI pilots before 2019 have not 
been able to deploy even a single application in production.1 Algorit

### Considerations

In short, the Refine method for text summarization with LLMs can pull in more relevant context and may be less lossy than Map Reduce. However, it requires many more calls to the LLM than Stuffing, and these calls are not independent, meaning they cannot be parallelized. Additionally, there is some potential dependency on the ordering of the documents. Latest documents they might become more relevant as this method suffers from recency bias.

## Conclusion


In this notebook you learn about different techniques to summarize long documents with LangChain and PaLM API. What you have seen in this notebook are only some of the possibilities you have. For example, there is another method called the Map-Rerank method which involves running an initial prompt on each chunk of data, which not only tries to complete a task but also gives a score for how certain it is in its answer. The responses are then ranked according to this score, and the highest score is returned.

With that being said, it is important to highlight that depending on your needs you may consider to use pure Foundational model with a custom framework to build generative ai application.

Here are some of the benefits of using a foundational model with a custom framework:

 - More flexibility to implement your application with different LLMs, prompting templates, document handling strategies and more.

 - More control to customize your generative applications based on your scenario.

 - Better performance to improve latency and scalability of your application.
